## Embeddings

In our previous example, we operated on high-dimensional bag-of-words vectors with length `vocab_size`, and we were explicitly converting from low-dimensional positional representation vectors into sparse one-hot representation. This one-hot representation is not memory-efficient, in addition, each word is treated independently from each other, i.e. one-hot encoded vectors do not express any semantic similarity between words.

In this unit, we will continue exploring **News AG** dataset. To begin, let's load the data and get some definitions from the previous unit.


In [ ]:
import sys
!{sys.executable} -m pip install --user --quiet tensorflow_datasets
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()


### What is embedding?

The idea of **embedding** is to represent words by lower-dimensional dense vectors, which somehow reflect semantic meaning of a word. We will later discuss how to build meaningful word embeddings, but for now let's just think of embeddings as a way to lower dimensionality of a word vector. 

So, embedding layer would take a word as an input, and produce an output vector of specified `embedding_size`. In a sense, it is very similar to `Dense` layer, but instead of taking one-hot encoded vector, it will be able to take a word number as an input.

By using embedding layer as a first layer in our network, we can switch from bag-or-words to **embedding bag** model, where we first convert each word in our text into corresponding embedding, and then compute some aggregate function over all those embeddings, such as `sum`, `average` or `max`.  

![Image showing an embedding classifier for five sequence words.](./images/embedding-classifier-example.png)

Our classifier neural network will consist of the following layers:

* `TextVectorization` layer, which will take a string as an input, and produce the tensor of token numbers. We will specify some reasonable vocabulary size `vocab_size`, and ignore all less-frequent tokens. Input shape will be 1, output shape - some number of tokens $n$, each of them containing numbers from 0 to `vocab_size`.
* `Embedding` layer, that will take $n$ numbers, and reduce each number to a dense vector of a given length (100 in our example). Thus, input tensor of shape $n$ will be transformed into $n\times 100$ tensor. 
* Aggregation layer, that will average out this tensor along the first axis, i.e. it will compute the average of all $n$ input tensors corresponding to different words. To implement this layer, we will use `Lambda` layer, and pass the function to compute the average. The output will have shape of 100, and it will be the numberic representation of the whole input sequence.
* Final `Dense` linear classifier

In [2]:
vocab_size = 50000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,)),
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         5000000   
_________________________________________________________________
lambda (Lambda)              (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 5,000,404
Trainable params: 5,000,404
Non-trainable params: 0
_________________________________________________________________


In the `summary` printout, in the **output shape** column, first tensor dimension `None` corresponds to minibatch size, second - to the length of token sequence. All token sequences in minibatch have different length, we will discuss how to deal with it in the next section.

Now let's train the network:

In [3]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
model.layers[0].adapt(ds_train.map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training vectorizer
938/938 [==============================] - 17s 17ms/step - loss: 0.7375 - acc: 0.8390 - val_loss: 0.3887 - val_acc: 0.8851


### Dealing with variable sequence size

Let's understand how training happens in minibatches. In the example above, input tensor has dimension of 1, and we use 128-long minibatches, so that actual size of the tensor is 128x1. However, number of tokens in each sentence is different. If we apply  `TextVectorization` layer to one input text, the number of tokens returned is different, depending on how text is tokenized:

In [4]:
vectorizer = model.layers[0]
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([12868    43], shape=(2,), dtype=int64)
tf.Tensor([  372  1614 11736     3   630   164], shape=(6,), dtype=int64)


However, when we apply vectorizer to a number of sequences, it has to produce a tensor of some rectangular shape, so it fills unused elements with PAD token (in our case - zero):

In [5]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[12868,    43,     0,     0,     0,     0],
       [  372,  1614, 11736,     3,   630,   164]])>

Those zeros are likely to be transformed by embedding into something not meaningful, which will not affect the result after averaging.

In [6]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 0.02457925, -0.00751474, -0.01965403, ...,  0.03359249,
         -0.00398868,  0.04344466],
        [-0.05184938, -0.19518852, -0.03138314, ...,  0.20190504,
          0.06465156, -0.05753793],
        [ 0.01857489,  0.01811096,  0.0308064 , ..., -0.01436602,
          0.05420632, -0.01932808],
        [ 0.01857489,  0.01811096,  0.0308064 , ..., -0.01436602,
          0.05420632, -0.01932808],
        [ 0.01857489,  0.01811096,  0.0308064 , ..., -0.01436602,
          0.05420632, -0.01932808],
        [ 0.01857489,  0.01811096,  0.0308064 , ..., -0.01436602,
          0.05420632, -0.01932808]],

       [[ 0.05496527, -0.16721806, -0.10409598, ...,  0.15814178,
         -0.0927898 ,  0.06700658],
        [ 0.06176873, -0.09997078, -0.12371238, ...,  0.08552848,
         -0.11854385,  0.14473796],
        [-0.00488436, -0.07180077,  0.0247047 , ...,  0.08314355,
         -0.01805916,  0.01055728],
        [ 0.00624194,  0.1058992 ,  0.03152668, ..., -0.08280457,
         -0.04

As you can see, the values of vectors corresponding to 0's are pretty close to 0 as well.

> **Note**: To minimize the amount of padding, in some cases it makes sense to sort all sequences in the dataset in the order of increasing length (or, more precisely, number of tokens). This will ensure that each minibatch contains sequences of similar length.


## Semantic Embeddings: Word2Vec

In our previous example, the model embedding layer learnt to map words to vector representation, however, this representation did not have much semantical meaning. It would be nice to learn such vector representation, that similar words or symonims would correspond to vectors that are close to each other in terms of some vector distance (eg. euclidian distance).

To do that, we need to pre-train our embedding model on a large collection of text in a specific way. One of the first ways to train semantic embeddings is called [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). It is based on two main architectures that are used to produce a distributed representation of words:

 - **Continuous bag-of-words** (CBoW) — in this architecture, we train the model to predict a word from surrounding context. Given the ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, the goal of the model is to predict $W_0$ from $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** is opposite to CBoW. The model uses surrounding window of context words to predict the current word.

CBoW is faster, while skip-gram is slower, but does a better job of representing infrequent words.

![Image showing both CBoW and Skip-Gram algorithms to convert words to vectors.](./images/example-algorithms-for-converting-words-to-vectors.png)

To experiment with word2vec embedding pre-trained on Google News dataset, we can use **gensim** library. Below we find the words most similar to 'neural'

> **Note:** When you first create word vectors, downloading them can take some time!

In [7]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [8]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


We can also extract vector embeddings from the word, to be used in training classification model (we only show first 20 components of the vector for clarity):

In [9]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Great thing about semantical embeddings is that you can manipulate vector encoding to change the semantics. For example, we can ask to find a word, whose vector representation would be as close as possible to words *king* and *woman*, and as far away from the word *man*:

In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Both CBOW and Skip-Grams are “predictive” embeddings, in that they only take local contexts into account. Word2Vec does not take advantage of global context. 

**FastText**, builds on Word2Vec by learning vector representations for each word and the charachter n-grams found within each word. The values of the representations are then averaged into one vector at each training step. While this adds a lot of additional computation to pre-training it enables word embeddings to encode sub-word information. 

Another method, **GloVe**, leverages the idea of co-occurence matrix, uses neural methods to decompose co-occurrence matrix into more expressive and non linear word vectors.

You can play with the example by changing embeddings to FastText and GloVe, since gensim supports those as well. Let's play with GloVe embedding:

In [11]:
glove = api.load('glove-wiki-gigaword-50')

An interesting thing about embeddings is that you can also manipulate embedding vectors to change the meaning of the word. Let's come back to the example above with king and queen. We will calculate the vector corrsponding to **KING-MAN+WOMAN**, and then find the closest word in the dictionary to that vector:

In [12]:
# get the vector corresponding to kind-man+woman
qvec = glove.word_vec('king')-glove.word_vec('man')+1.2*glove.word_vec('woman')
# find the index of the closest embedding vector 
d = tf.reduce_sum((glove.vectors-qvec)**2,axis=1)
min_idx = tf.argmin(d).numpy()
# find the corresponding word
glove.index2word[min_idx]

'queen'

> **NOTE**: We had to add a small coefficient of 1.2 to *woman* vector - try removing it to see what happens.

To find the closest vector, we use TensorFlow machinery to compute a vector of distances between our vector and all vectors in the vocabulary, and then find the index of minimal word using `argmin`.

## Using Pre-Trained Embeddings in Keras

We can modify the example above to pre-populate the matrix in our embedding layer with semantical embeddings, such as Word2Vec. We need to take into account that vocabularies of pre-trained embedding and our text corpus will likely not match, and we need to switch to one vocabulary. There are two possible options: use tokenizer vocabulary, and use vocabulary from Word2Vec embeddings.

### Using Tokenizer vocabulary

When using tokenizer vocabulary, some of the words from the vocabulary will have corresponding Word2Vec embeddings, and some will be missing. Given that our vocabulary size is `vocab_size`, and Word2Vec embedding vector length is `embed_size`, embedding layer will be repesented by weight matrix of shape `vocab_size`$\times$`embed_size`. We will populate this matrix by going through the vocabulary:

In [13]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 31368 words, 18632 words missing


For the words that are not present in the Word2Vec vocabulary, we can either leave them as zeroes, or generate a random vector.

Now we can define embedding layer of the corresponsing size with pre-trained weights:

In [14]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

Now let's train our model. Note that the time it takes to train the model is significantly larger than in the previous example, due to larger embedding layer size, and thus much higher number of parameters. Also, because of this, we may need to train our model on more examples if we want to avoid overfitting.

In [15]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 7s 7ms/step - loss: 1.0271 - acc: 0.8221 - val_loss: 0.7979 - val_acc: 0.8455


> **Note**: The accuracy is likely slightly lower, but the training goes very fast, because we are not re-training embedding layer. You may want to experiment with embedding training and see if you can get higher accuracy. In this case, it makes sense to intialize missing words with small non-zero values by uncommenting corresponding line in the matrix population function.

### Using Embedding Vocabulary

Another reason we do not see huge increase in accuracy is likely due to quite different vocalularies. 
To overcome the problem of different vocabularies, we can use one of the following solutions:
* Re-train word2vec model on our vocabulary
* Load our dataset with the vocabulary from the pre-trained word2vec model. Vocabulary used to load the dataset can be specified during loading.

The latter approach seems easiter. First of all, we will create `TextVectorization` layer with specified vocabulary, taken from GloVe word embeddings:

In [16]:
vocab = list(w2v.vocab.keys())
# vectorizer = keras.layers.experimental.preprocessing.TextVectorization(vocabulary=vocab,input_shape=(1,))
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

GenSim word embeddings library contains convenient function, `get_keras_embeddings`, which will automatically create corresponding Keras embeddings layer for you.

In [17]:
# Sometimes, to make this work on a newer TensorFlow version, you may need to execute the following code snippet
try: glove.get_keras_embedding()
except: pass

In [18]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 8s 8ms/step - loss: 1.3390 - acc: 0.4984 - val_loss: 1.3001 - val_acc: 0.5733
Epoch 2/5
938/938 [==============================] - 7s 7ms/step - loss: 1.2596 - acc: 0.5725 - val_loss: 1.2343 - val_acc: 0.5846
Epoch 3/5
938/938 [==============================] - 7s 7ms/step - loss: 1.1987 - acc: 0.5892 - val_loss: 1.1829 - val_acc: 0.5946
Epoch 4/5
938/938 [==============================] - 7s 7ms/step - loss: 1.1508 - acc: 0.6003 - val_loss: 1.1418 - val_acc: 0.6034
Epoch 5/5
938/938 [==============================] - 7s 7ms/step - loss: 1.1125 - acc: 0.6095 - val_loss: 1.1084 - val_acc: 0.6100


One of the reasons we are not seeing significant increase in accuracy is due to the fact that some words from our dataset are missing in the pre-trained GloVe vocabulary, and thus they are essentially ignored. To overcome this fact, we can train our own embeddings on our dataset. 


## Training your own embeddings

In our examples, we have been using pre-trained semantic embeddings, but it is interesting to see how those embeddings can be trained using either CBoW, or Skip-gram architectures. This exercise goes beyond this module, but those interested might want to check out this [official TensorFlow tutorial on training Word2Vec model](https://www.tensorflow.org/tutorials/text/word2vec). Also, **gensim** framework can be used to train most commonly used embeddings in a few lines of code, as described [in official documentation](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#training-your-own-model).

## Contextual Embeddings

One key limitation of tradition pretrained embedding representaitons such as Word2Vec is the problem of word sense disambigioution. While pretrained embeddings can capture some of the meaning of words in context, every possible meaning of a word is encoded into the same embedding. This can cause problems in downstream models, since many words such as the word 'play' have different meanings depending on the context they are used in.

For example word 'play' in those two different sentences have quite different meaning:
- I went to a **play** at the theature.
- John wants to **play** with his friends.

The pretrained embeddings above represent both of these meanings of the word 'play' in the same embedding. To overcome this limitation, we need to build embeddings based on the **language model**, which is trained on a large corpus of text, and *knows* how words can be put together in different contexts. Discussing contextual embeddings is out of scope for this tutorial, but we will come back to them when talking about language models in the next unit.
